<a href="https://colab.research.google.com/github/chiragraokv/MachineLearning/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from PIL import Image
import pandas as pd
import os

In [3]:
class Layers_hidden:
    def __init__(self,Weights_no,neuron_no):
        self.weights = np.random.randn(neuron_no, Weights_no)
        self.bias = np.random.randn(neuron_no)

    def ReLU(self,x):
        return np.maximum(x,0)

    def Process(self,input):
        self.input = input
        self.y = np.dot(self.weights,self.input) + self.bias  #gives an array of output with no of elements  = no of neuron
        self.z = self.ReLU(self.y) #activation function based of the layer being used
        self.output_layer_hidden = self.z
        return self.z

    def Backward_propogation_hidden(self,dvalue,learning_rate):
        self.dw = np.dot(dvalue,self.input)
        self.dx = np.dot(dvalue,self.weights)
        self.db = dvalue
        self.update_parameters(learning_rate)
        return np.average(self.dx)

    def update_parameters(self,learning_rate):
        self.weights = self.weights +learning_rate*self.dw
        self.bias = self.bias + learning_rate*self.db

class Layer_output:
    def __init__(self,Weights_no,neuron_no):
        self.weights = np.random.randn(neuron_no, Weights_no)
        self.bias = np.random.randn(neuron_no)

    def Softmax(self,x):
        x = x-np.max(x)
        return ((np.exp(x))/np.sum(np.exp(x)))

    def Process(self,input):
        self.input = input
        self.y = np.dot(self.weights,self.input) + self.bias  #gives an array of output with no of elements  = no of neuron this is for output layer
        self.z = self.Softmax(self.y)
        self.output_layer = self.z
        return self.z

    def Cross_entropy_loss(self,Y_actual):
        return (-np.sum(Y_actual*(np.log(self.output_layer + 1e-8))))

    def Backward_propogation_output(self,yactual,learning_rate):

        self.dw = np.dot((yactual - self.output_layer),self.input)
        self.dx = np.dot((yactual - self.output_layer),self.weights)
        self.db = (yactual - self.output_layer)
        self.update_parameters(learning_rate)
        return np.average(self.dx)

    def update_parameters(self,learning_rate):
        self.weights = self.weights + learning_rate*self.dw
        self.bias = self.bias + learning_rate*self.db


In [4]:
class Train_Test_Neural_network:
    def __init__(self,layer_info,X_train,Y_train,X_test,Y_test):
        self.no_Layers = len(layer_info)-1
        self.Layer_size = layer_info
        self.Layer = []
        self.loss_in_one_itiration = 0
        self.Loss = []
        self.X_train = X_train
        self.Y_train= Y_train
        self.X_train_scrap = X_train[0, : ]
        self.X_test = X_test
        self.Y_test = Y_test
        self.dw = 0
        self.dx =0
        self.db = 0
        for i in range(self.no_Layers):
            self.Layer.append(Layers_hidden(len(self.X_train_scrap),self.Layer_size[i])) #making of object list and inisializing parameters.
            self.X_train_scrap = self.Layer[i].Process(self.X_train_scrap)
        self.Layer_last = Layer_output(len(self.X_train_scrap),self.Layer_size[i+1])
        print("command out of initialization\n")

    def execute_through_layers(self,X):
        for i in range(self.no_Layers):
            X = self.Layer[i].Process(X)
        self.X_last = self.Layer_last.Process(X)

    def gradient_descent(self,Y_test):
        self.Y_test_1 = Y_test
        for i in range(self.no_Layers):
            self.dx_last_layer = self.Layer_last.Backward_propogation_output(self.Y_test_1,self.learning_rate)
            for j in range(self.no_Layers):
                self.dx = self.Layer[i].Backward_propogation_hidden(self.dx_last_layer,self.learning_rate)

    def Train_layers(self,itirations,learning_rate):
        self.learning_rate = learning_rate
        for i in range(itirations):
            for j in range(self.X_train.shape[0]):
                self.execute_through_layers(self.X_train[j]) # this function is called to execute through layers each time
                self.loss_in_one_itiration = self.loss_in_one_itiration + (self.Layer_last.Cross_entropy_loss(self.Y_train[j]))  # to save the Cross entropy loss in each itiration
                self.gradient_descent(self.Y_test[j])
            self.loss_in_one_itiration = self.loss_in_one_itiration/self.X_train.shape[0]
            self.Loss.append(self.loss_in_one_itiration)
            self.loss_in_one_itiration = 0


In [5]:
class One_hot_encodeing():
    def __init__(self,data,categories):
        self.categories = categories
        self.data = data
        self.Y_one_hot = np.zeros(shape=(data.shape[0],self.categories.shape[0]))

    def encode(self):
        for index,cat in enumerate(self.data[:,-1]):
            index_cat = np.where(cat == self.categories)
            self.Y_one_hot[index][index_cat] = 1
        return self.Y_one_hot

In [14]:
class Image_Extraction_from_file:
    def __init__(self,path_folder):
        self.path_folder = path_folder
        self.image_data = np.empty((28*28))

    def Get_images(self,image_path):

        image = Image.open(image_path)
        image_array = np.array(image.getdata(),dtype=np.int32)/256
        #append the image data into image data array which will contain all the image data with the category lable
        self.image_data.append(image_array)

    def Extract_from_folder(self):
        category_name = []
        self.image_data = []
        for root, dirs, files in os.walk(self.path_folder):
            for name in files:
                if name.endswith(".png"):
                    image_path = os.path.join(root, name)
                    self.Get_images(image_path)
                    subdir_name = os.path.basename(root)
                    category_name.append(subdir_name)

        self.image_data = np.array(self.image_data)
        category_name = np.array(category_name).reshape(-1, 1)
        unique_cat_list = np.unique(category_name)
        # Combine image data with labels
        self.image_data = np.hstack((self.image_data, category_name))
        encode_one_hot = One_hot_encodeing(self.image_data,unique_cat_list)
        self.Y_one_hot = encode_one_hot.encode()
        return self.image_data[:,:-1].astype(float),self.Y_one_hot

In [15]:
path = "C:\\CSE\\Chirag-Rao-KV\\Task 4.0\\Train"
Test_file = Image_Extraction_from_file(path)

In [17]:
X_images,Y_images = Test_file.Extract_from_folder()


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [11]:
Layer_info = np.array([784,10,10,5,5])

In [12]:
Layer_info

array([784,  10,  10,   5,   5])

In [ ]:
Neural_network = Train_Test_Neural_network(Layer_info,X_images,Y_images,X_images,Y_images)
Neural_network.Train_layers(1,0.001)